# part0: imports

In [ ]:
import os, sys, pathlib
from pprint import pprint
import gc
import pickle
from importlib import reload
import logging, warnings
# logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import PCA
import scipy.linalg as linalg

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
from matplotlib.collections import LineCollection
%matplotlib inline


import pyaldata as pyal

try:
    nbPath = pathlib.Path.cwd()
    RepoPath = nbPath.parent
    os.chdir(RepoPath)

    from tools import utilityTools as utility
    from tools import dataTools as dt
    import params
    monkey_defs = params.monkey_defs
    mouse_defs = params.mouse_defs

    set_rc =  params.set_rc_params
    set_rc()
    root = params.root
    
    os.chdir(RepoPath / 'monkey')
    %run "_dataset-selection.ipynb"

finally:
    os.chdir(nbPath)

print('Done')

---

#### Load the lower bound values for the canonical correlations

takes a couple of minutes to execute.

In [ ]:
%run "_lower-bound.ipynb"

---

#### Load the upper bound values for the canonical correlations

takes a couple of minutes to execute.

In [ ]:
%run "_upper-bound.ipynb"



---

back to the original idea:

1. this:
> # Compare different epochs
> the idea is to see whether canonical axes between 2 animals provide a higher VAF for time epochs in the trial that they have not been trained on, compared to, for example, M1-PMd axes in a single animal.

2. and a similar but different idea:
> two monkeys during the same epoch vs. one monkey between 2 different epochs.
i.e., preparation vs execution, ...

#### Try the second idea first:

# CCA comparison

## 2 monkeys, same epoch

load the data

In [ ]:
full_list_MCx = []
for animal, sessionList in GoodDataList[defs.areas[2]].items():
    if 'Mr' in animal:
        continue  # to remove MrT
    full_list_MCx.append((animal,sessionList))
full_list_MCx = [(animal,session) for animal,sessions in full_list_MCx for session in set(sessions)]
# load the DFs
warnings.filterwarnings("ignore")
allDFs_MCx = []
for animal, session in full_list_MCx:
    path = root/animal/session
    allDFs_MCx.append(defs.prep_general(dt.load_pyal_data(path)))


warnings.filterwarnings("default")


pairFileList1 = []
for I, (animal1,session1) in enumerate(full_list_MCx):
    for J, (animal2,session2) in enumerate(full_list_MCx):
        if J<=I or animal1 == animal2: continue  # to repetitions
        if 'Chewie' in animal1 and 'Chewie' in animal2: continue 
        pairFileList1.append((I,J))

print(f'{len(pairFileList1)=}')

collecting all the data in a matrix, `AllData`: $sessions \times targets \times  trials \times time \times PCs$

In [ ]:
side1df = [allDFs_MCx[i] for i,_ in pairFileList1]
side2df = [allDFs_MCx[j] for _,j in pairFileList1]
warnings.filterwarnings("ignore")
AllData1 = dt.get_data_array(side1df, defs.exec_epoch, area=defs.areas[2], model=defs.n_components)
AllData2 = dt.get_data_array(side2df, defs.exec_epoch, area=defs.areas[2], model=defs.n_components)
_,_, min_trials, min_time,_ = np.min((AllData1.shape,AllData2.shape),axis=0)
print(f'{min_trials=}\n{min_time=}')

warnings.filterwarnings("default")

allCCs0=[]
for sessionData1,sessionData2 in zip(AllData1,AllData2):
    data1 = np.reshape(sessionData1[:,:min_trials,:min_time,:], (-1,defs.n_components))
    data2 = np.reshape(sessionData2[:,:min_trials,:min_time,:], (-1,defs.n_components))
    allCCs0.append(dt.canoncorr(data1, data2))
allCCs0 = np.array(allCCs0).T

# plot
_,ax = plt.subplots(dpi=100)
utility.shaded_errorbar(ax, allCCs0, color='b', marker = 'o', label=f'{defs.areas[2]} $n={allCCs0.shape[1]}$ sessions')
ax.set_ylim([0,1])
ax.set_xlabel('components')
ax.legend()
ax.set_ylabel('canonical correlation')
ax.set_title('CCA --- across monkey', usetex=True);

_,ax = plt.subplots(ncols=1, figsize=(10,5))
ax.plot(allCCs0[:4,:].mean(axis=0),'.')
ax.set_ylabel('average canonical correlation')
ax.set_title(f'CCA --- across monkey {defs.areas[2]} ', usetex=True);
ax.set_ylim([0,1])
ax.set_xlabel('sessions')


### Overall Figures

across monkey and within monkey together

In [ ]:
# plot
fig,ax = plt.subplots(ncols=1, figsize=(3,3))

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), allCCs0, color='b', marker = 'o', label=f'Across, $n={allCCs0.shape[1]}$')

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), CC_upper_bound_MCx, color='cornflowerblue', marker = '<', ls='--', label=f'Within, $n={CC_upper_bound_MCx.shape[1]}$')

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), CC_lower_bound_MCx, color='gray', marker = '>', ls=':', label=f'Control, $n={CC_lower_bound_MCx.shape[1]}$')


ax.set_ylim([0,1])
ax.set_xlim([.5,defs.n_components+.5])
ax.set_xlabel('Neural mode')
ax.set_title(f'{defs.areas[2]} Alignment')
ax.legend(loc=(.35,.7))
ax.set_ylabel('Canonical correlation')
fig.tight_layout()

fig.savefig(params.figPath / 'monkey-cca-modes.pdf', format='pdf', bbox_inches='tight')

comapring group correlations
>Like the NN paper, _Fig. 4e_.

In [ ]:
fig,ax = plt.subplots(ncols=1, figsize=(3,3))
bins = np.arange(0,1,0.05)

ax.hist(allCCs0[:4,:].mean(axis=0), bins=bins, density=True,
        color='b', alpha=.8, label=f'across, $n={allCCs0[:4,:].mean(axis=0).shape[0]}$')
ax.hist(CC_upper_bound_MCx[:4,:].mean(axis=0), bins=bins, density=True,
        color='cornflowerblue', alpha=.8, label=f'within, $n={CC_upper_bound_MCx[:4,:].mean(axis=0).shape[0]}$')
ax.hist(CC_lower_bound_MCx[:4,:].mean(axis=0), bins=bins, density=True,
        color='gray', alpha=.8, label=f'control, $n={CC_lower_bound_MCx[:4,:].mean(axis=0).shape[0]}$')

ax.set_title(f'Average of top 4 CCs in {defs.areas[2]}')
ax.set_xlim([0,1])
ax.set_xlabel('Canonical correlation')
ax.set_ylabel('Probability')
# ax.set_yticks([])
ax.legend(loc=2)

fig.savefig(params.figPath / 'monkey-cca-dist.pdf', format='pdf', bbox_inches='tight')

# For Preparation epoch

---

#### Load the upper bound values for the canonical correlations

takes a couple of minutes to execute.

In [ ]:
%run "_upper-bound-prep.ipynb"

In [ ]:
side1df = [allDFs_MCx[i] for i,_ in pairFileList1]
side2df = [allDFs_MCx[j] for _,j in pairFileList1]
warnings.filterwarnings("ignore")
AllData1 = dt.get_data_array(side1df, defs.prep_epoch, area=defs.areas[2], model=defs.n_components)
AllData2 = dt.get_data_array(side2df, defs.prep_epoch, area=defs.areas[2], model=defs.n_components)
_,_, min_trials, min_time,_ = np.min((AllData1.shape,AllData2.shape),axis=0)
print(f'{min_trials=}\n{min_time=}')

warnings.filterwarnings("default")

allCCs1=[]
for sessionData1,sessionData2 in zip(AllData1,AllData2):
    data1 = np.reshape(sessionData1[:,:min_trials,:min_time,:], (-1,defs.n_components))
    data2 = np.reshape(sessionData2[:,:min_trials,:min_time,:], (-1,defs.n_components))
    allCCs1.append(dt.canoncorr(data1, data2))
allCCs1 = np.array(allCCs1).T

# plot

_,ax = plt.subplots(dpi=100)
utility.shaded_errorbar(ax, allCCs1, color='b', marker = 'o', label=f'{defs.areas[2]} $n={allCCs1.shape[1]}$ sessions')
ax.set_ylim([0,1])
ax.set_xlabel('components')
ax.legend()
ax.set_ylabel('canonical correlation')
ax.set_title('CCA --- across monkey')

_,ax = plt.subplots(ncols=1, figsize=(10,5), dpi=100)
ax.plot(allCCs1[:4,:].mean(axis=0),'.')
ax.set_ylabel('average canonical correlation')
ax.set_title(f'CCA --- across monkey {defs.areas[2]}');
ax.set_ylim([0,1])
ax.set_xlabel('sessions')


# Overall PREP epoch Figures

In [ ]:
# plot
fig,ax = plt.subplots(ncols=1, figsize=(3,3))

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), allCCs1, color='b', marker = 'o', label=f'Across, $n={allCCs1.shape[1]}$')

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), CC_upper_bound_MCx, color='cornflowerblue', marker = '<', ls='--', label=f'Within, $n={CC_upper_bound_MCx.shape[1]}$')

utility.shaded_errorbar(ax, np.arange(1,defs.n_components+1), CC_lower_bound_MCx, color='gray', marker = '>', ls=':', label=f'Control, $n={CC_lower_bound_MCx.shape[1]}$')


ax.set_ylim([0,1])
ax.set_xlim([.5,defs.n_components+.5])
ax.set_xlabel('Neural mode')
ax.set_title(f'{defs.areas[2]} preparation alignment')
ax.legend(loc=(.35,.7))
ax.set_ylabel('Canonical correlation')
fig.tight_layout()

fig.savefig(params.figPath / 'monkey-prep-cca-modes.pdf', format='pdf', bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(ncols=1, figsize=(3,3))
bins = np.arange(0,1,0.05)

ax.hist(allCCs1[:4,:].mean(axis=0), bins=bins, density=True,
        color='b', alpha=.8, label=f'Across, $n={allCCs1[:4,:].mean(axis=0).shape[0]}$')
ax.hist(CC_upper_bound_MCx[:4,:].mean(axis=0), bins=bins, density=True,
        color='cornflowerblue', alpha=.8, label=f'Within, $n={CC_upper_bound_MCx[:4,:].mean(axis=0).shape[0]}$')
ax.hist(CC_lower_bound_MCx[:4,:].mean(axis=0), bins=bins, density=True,
        color='gray', alpha=.8, label=f'Control, $n={CC_lower_bound_MCx[:4,:].mean(axis=0).shape[0]}$')

ax.set_title(f'Top 4 CCs during prep. --- {defs.areas[2]}')
ax.set_xlim([0,1])
ax.set_xlabel('Canonical correlation')
ax.set_ylabel('Probability')
# ax.set_yticks([])
ax.legend(loc=2)

fig.savefig(params.figPath / 'monkey-prep-cca-dist.pdf', format='pdf', bbox_inches='tight')

---

## one monkey, 2 epochs

as an extra control

In [ ]:
single_FileList1 = []
for I, _ in enumerate(full_list_M1):
    single_FileList1.append(I)

print(f'{len(single_FileList1)=}')

single_FileList2 = []
for I, (animal1,session1) in enumerate(full_list_PMd):
    single_FileList2.append(I)

print(f'{len(single_FileList2)=}')

In [ ]:
warnings.filterwarnings("ignore")
side1df = [allDFs_M1[i] for i in single_FileList1]

AllData1 = dt.get_data_array(side1df, prep_epoch, area=areas[0], model=n_components)
AllData2 = dt.get_data_array(side1df, exec_epoch, area=areas[0], model=n_components)
_,_, min_trials, min_time,_ = np.min((AllData1.shape,AllData2.shape),axis=0)
print(f'{min_trials=}\n{min_time=}')

# PMd
side2df = [allDFs_PMd[j] for j in single_FileList2]

AllData1_ = dt.get_data_array(side2df, prep_epoch, area=areas[1], model=n_components)
AllData2_ = dt.get_data_array(side2df, exec_epoch, area=areas[1], model=n_components)
_,_, min_trials_, min_time_,_ = np.min((AllData1_.shape,AllData2_.shape),axis=0)
print(f'{min_trials_=}\n{min_time_=}')

inCCs0=[]
for sessionData1,sessionData2 in zip(AllData1,AllData2):
    data1 = np.reshape(sessionData1[:,:min_trials,:min_time,:], (-1,n_components))
    data2 = np.reshape(sessionData2[:,:min_trials,:min_time,:], (-1,n_components))
    inCCs0.append(dt.canoncorr(data1, data2))
inCCs0 = np.array(inCCs0).T

inCCs1=[]
for sessionData1,sessionData2 in zip(AllData1_,AllData2_):
    data1 = np.reshape(sessionData1[:,:min_trials_,:min_time_,:], (-1,n_components))
    data2 = np.reshape(sessionData2[:,:min_trials_,:min_time_,:], (-1,n_components))
    inCCs1.append(dt.canoncorr(data1, data2))
inCCs1 = np.array(inCCs1).T
warnings.filterwarnings("default")

# plotting
_,ax = plt.subplots()

utility.shaded_errorbar(ax, inCCs0, color='b', marker = 'o', label=f'{areas[0]} $n={inCCs0.shape[1]}$ sessions')
utility.shaded_errorbar(ax, inCCs1, color='r', marker = 'o', label=f'{areas[1]} $n={inCCs1.shape[1]}$ sessions')
ax.set_ylim([0,1])
ax.set_xlabel('components')
ax.legend()
ax.set_ylabel('canonical correlation')
ax.set_title('CCA --- within monkey', usetex=True);

fig,axes = plt.subplots(ncols=2, figsize=(10,5))
axes[0].plot(inCCs0[:4,:].mean(axis=0),'.')
axes[1].plot(inCCs1[:4,:].mean(axis=0),'.')
axes[0].set_ylabel('average canonical correlation')
for i, ax in enumerate(axes):
    ax.set_title(f'CCA --- within monkey {areas[i]} ', usetex=True);
    ax.set_ylim([0,1])
    ax.set_xlabel('sessions')

---

## one monkey, 2 areas

as an extra control

In [ ]:
full_list_dual = []
for animal, sessionList in GoodDataList[areas[0]].items():
    if 'Mr' in animal:
        continue  # to remove MrT
    full_list_M1.append((animal,sessionList))
full_list_M1 = [(animal,session) for animal,sessions in full_list_M1 for session in set(sessions)]
# load the DFs
warnings.filterwarnings("ignore")
allDFs_M1 = []
for animal, session in full_list_M1:
    path = root/animal/session
    allDFs_M1.append(prep_general(dt.load_pyal_data(path)))


# Part 2
Plot figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params({'axes.labelsize':'x-small'})
    figsize=(7,7)
    fig=plt.figure(figsize=figsize,dpi=600)
    
   

    ##########################################
    # 1: task rules
    gs1   =fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0.6, right=.2, top=.74)
    ax1   =fig.add_subplot(gs1[0])
    gs1_   =fig.add_gridspec(nrows=1, ncols=1, left=.25, bottom=0.64, right=.39, top=.74)
    ax1_   =fig.add_subplot(gs1_[0])
    gs1__   =fig.add_gridspec(nrows=1, ncols=2, left=.43, bottom=0.64, right=.64, top=.74, wspace=.1)
    ax1__l  =fig.add_subplot(gs1__[0])
    ax1__r  =fig.add_subplot(gs1__[1])
    gs1___l   =fig.add_gridspec(nrows=1, ncols=1, left=.675, bottom=0.64, right=.815, top=.74)
    ax1___l  =fig.add_subplot(gs1___l[0])
    gs1___r   =fig.add_gridspec(nrows=1, ncols=1, left=.86, bottom=0.64, right=1, top=.74)
    ax1___r  =fig.add_subplot(gs1___r[0])

    
    ax1.xaxis.set_visible(False)
    ax1.yaxis.set_visible(False)
    ax1.spines['bottom'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)

    
    plot_panel_A(ax1_,ax1__l,ax1__r,ax1___l,ax1___r)

    

    ##########################################
    # 2: DLS
    gs2= fig.add_gridspec(nrows=1, ncols=4, left=0, bottom=0.44, right=.6, top=.54)
    
    axes2=plot_pre_post_traj(root, gs2, animalList2Ctrl, profile2pre, profile2post, sessionIndex1pre, sessionIndex2post)
    
    axes2[0].clear()
    plot_animal_image(axes2[0], animalList2Ctrl)
    axes2[0].set_ylabel('DLS')
    axes2[1].set_title('session #$-1$',fontsize='xx-small')
    axes2[2].set_title('Acute (#$+1$)',fontsize='xx-small')
    axes2[3].set_title(f'First stable (#$+{sessionIndex2post[-1]+1}$)',fontsize='xx-small')
    
    #adding the Before and After text and line
    y_top=gs2.top+.03
    add_underlined_text_centered_at(fig,p1=(axes2[1].get_position().bounds[0],y_top),p2=(gs2.right/4+axes2[1].get_position().bounds[2],y_top), s='Before')
    add_underlined_text_centered_at(fig,p1=(axes2[2].get_position().bounds[0],y_top),p2=(gs2.right,y_top), s='After')
    
    
    
    ##########################################
    # 3: DMS
    gs3= fig.add_gridspec(nrows=1, ncols=4, left=0, bottom=0.32, right=.6, top=.42)
    
    axes3=plot_pre_post_traj(root, gs3, animalList3Ctrl, profile3pre, profile3post, sessionIndex1pre, sessionIndex3post)
    
    axes3[0].clear()
    plot_animal_image(axes3[0], animalList3Ctrl)
    axes3[0].set_ylabel('DMS')
    axes3[3].set_title(f'(#$+{sessionIndex3post[-1]+1}$)',fontsize='xx-small')
    


    
    
    ##########################################
    # 4: DS
    gs4= fig.add_gridspec(nrows=1, ncols=4, left=0, bottom=0.2, right=.6, top=.3)
    
    axes4=plot_pre_post_traj(root, gs4, animalList4Ctrl, profile4pre, profile4post, sessionIndex1pre, sessionIndex4post, showText=True)
    
    axes4[0].clear()
    plot_animal_image(axes4[0], animalList4Ctrl)
    axes4[0].set_ylabel('DS')
    axes4[3].set_title(f'(#$+{sessionIndex4post[-1]+1}$)',fontsize='xx-small')

    #add text and line
    x_=np.mean([gs4.right,gs4.left])
    fig.add_artist(mpl.lines.Line2D([x_,x_] , [gs4.bottom,gs2.top],
                                    linestyle=':',lw=.7,color='gray', zorder=1))
   
    fig.add_artist(mpl.text.Text(x=x_, y=gs2.top,
                                        text=' Lesion', c='k',
                                        ha='center',va='bottom',fontsize='xx-small',rotation=90))


    
    ##########################################
    # 5: % entrance time time course
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=0.67, bottom=.49, right=1, top=.59)
    ax5= fig.add_subplot(gs5[0])
    
    #DLS
    xDLS5=plot_event_rec(root, ax5, ProfilesDLS5,TaskParamToPlot=TaskParamToPlot5,
                         nPre=nPre5, nPost=nPost5, color=DLSColor,shift=ShiftDLS5)

    #DMS
    xDMS5=plot_event_rec(root, ax5, ProfilesDMS5,TaskParamToPlot=TaskParamToPlot5,
                         nPre=nPre5, nPost=nPost5, color=DMSColor,shift=ShiftDMS5)

    #DS
    xDS5 =plot_event_rec(root, ax5, ProfilesDS5,TaskParamToPlot=TaskParamToPlot5,
                         nPre=nPre5, nPost=nPost5, color=DSColor,shift=ShiftDS5)

    
    add_legend_for_lesion_types(ax5,(len(xDLS5),len(xDMS5),len(xDS5)))
    
    ax5.axhline(y=7, ls=':',c='gray',lw=1, zorder=-5)
    ax5.set_ylim([3,9])
    ax5.spines['left'].set_bounds(3,9)
    ax5.set_yticks(np.arange(3,10))
    ax5.set_yticklabels([3,'','','',7,'',9])
    ax5.set_ylabel('Entrance time (s)')
    ax5.set_xticklabels([])

    plot_session_def(ax5,sessionSlices=(preSlice6,postSlice6,finSlice6))


    
    
    
    ##########################################
    # 6: ET event plot
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=0.67, bottom=.2, right=.80, top=.32)
    ax6= fig.add_subplot(gs6[0])
        
    data6,*_=plot_event_1on1(root, ax6, Profiles6, colorCode=ColorCode, badAnimals=None, x_pos=(1,2,3,4),
                         TaskParamToPlot=TaskParamToPlot5, nInit=initSlice6, nPre=preSlice6, nPost=postSlice6,nFin=finSlice6)


    ax6.set_ylim([2,11])
    ax6.spines['left'].set_bounds(2,10)
    ax6.set_yticks(np.arange(2,11))
    ax6.set_yticklabels([2,'','','','',7,'','',10])
    ax6.set_ylabel('Entrance time (s)')


    #STATS
    p6,s6=bootstrapTest(data6[:,-1]-data6[:,1])
    permtest.plotSigPair(ax6,y=10,x=(2,4), s=s6, color=colorSig,lw=.8)
    
    
    p6_,s6_=bootstrapTest(data6[:,0]-data6[:,1])
    permtest.plotSigPair(ax6,y=10.9,x=(1,2), s=s6_, color=colorSig,lw=.8)

    p6__,s6__=bootstrapTest(data6[:,1]-data6[:,2])
    permtest.plotSigPair(ax6,y=10.9,x=(2,3), s=s6__, color=colorSig,lw=.8)
    
    p6___,s6___=bootstrapTest(data6[:,2]-data6[:,3])
    permtest.plotSigPair(ax6,y=10.9,x=(3,4), s=s6___, color=colorSig,lw=.8)

    
    
    ##########################################
    # 7: % %Routine time course
    gs7= fig.add_gridspec(nrows=1, ncols=1, left=0.67, bottom=.37, right=1, top=.47)
    ax7= fig.add_subplot(gs7[0])
    
    #DLS
    xDLS7=plot_event_rec(root, ax7, ProfilesDLS5,TaskParamToPlot=TaskParamToPlot7,
                         nPre=nPre5, nPost=nPost5, color=DLSColor,shift=ShiftDLS5)

    #DMS
    xDMS7=plot_event_rec(root, ax7, ProfilesDMS5,TaskParamToPlot=TaskParamToPlot7,
                         nPre=nPre5, nPost=nPost5, color=DMSColor,shift=ShiftDMS5)

    #DS
    xDS7 =plot_event_rec(root, ax7, ProfilesDS5,TaskParamToPlot=TaskParamToPlot7,
                         nPre=nPre5, nPost=nPost5, color=DSColor,shift=ShiftDS5)

    
    plot_session_def(ax7,sessionSlices=(preSlice6,postSlice6))
    plot_session_def(ax7,sessionSlices=(finSlice6,))
    
    ax7.set_ylim([0,1])
    ax7.spines['left'].set_bounds(0,1)
    ax7.set_yticks(np.arange(0,1.1,.2))
    ax7.set_yticklabels([0]+['']*4+[100])
    ax7.set_ylabel('% Routine',labelpad=-3)
    ax7.set_xlabel('Session # relative to lesion', labelpad=0)
    
    
    x_=fig.transFigure.inverted().transform(ax7.transData.transform((0,0)))[0]
    fig.add_artist(mpl.lines.Line2D([x_,x_] , [ax7.get_position().bounds[1], ax5.get_position().get_points()[1][1]],
                                    linestyle=':',lw=.7,color='gray', zorder=1))

    
    ##########################################
    # 8: %Routine event plot
    gs8= fig.add_gridspec(nrows=1, ncols=1, left=0.87, bottom=.2, right=1, top=.32)
    ax8= fig.add_subplot(gs8[0])
    
    data8,*_=plot_event_1on1(root, ax8, Profiles6, colorCode=ColorCode, badAnimals=None, x_pos=(1,2,3,4),
                         TaskParamToPlot=TaskParamToPlot7, nInit=initSlice6, nPre=preSlice6, nPost=postSlice6,nFin=finSlice6)



    ax8.set_ylim([0,1.1])
    ax8.spines['left'].set_bounds(0,1)
    ax8.set_yticks(np.arange(0,1.1,.2))
    ax8.set_yticklabels([0]+['']*4+[100])
    ax8.set_ylabel('% Routine',labelpad=-2)
    
    
    #STATS
    p8,s8=bootstrapTest(data8[:,-1]-data8[:,1])
    permtest.plotSigPair(ax8,y=1,x=(2,4), s=s8, color=colorSig,lw=.8)

    p8_,s8_=bootstrapTest(data8[:,0]-data8[:,1])
    permtest.plotSigPair(ax8,y=1.09,x=(1,2), s=s8_, color=colorSig,lw=.8)

    p8__,s8__=bootstrapTest(data8[:,1]-data8[:,2])
    permtest.plotSigPair(ax8,y=1.09,x=(2,3), s=s8__, color=colorSig,lw=.8)
    
    p8___,s8___=bootstrapTest(data8[:,2]-data8[:,3])
    permtest.plotSigPair(ax8,y=1.09,x=(3,4), s=s8___, color=colorSig,lw=.8)


    
    

    
#     fig.align_ylabels([ax5,ax6,ax7])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax1,ax1_,ax1__l,ax1___r,axes2[0],axes3[0],axes4[0],ax5,ax7,ax6,ax8)
    OFFX=np.array([.02]*len(AXES))
    OFFY=np.array([.01]*len(AXES))
    
    OFFY[[-1,-2]]=0
    OFFX[[-1,-2]]=0.06
    OFFX[[-3,-4]]=0.06
    OFFX[[3,2,1]]=0.04
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Task_Example_Group.pdf'),
                format='pdf', bbox_inches='tight')
    
    thisPath  =os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Task_Example_Group.pdf')
    sketchPath=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','TreadmillSketch.pdf')
    if os.path.exists(sketchPath):
        f1=ppdf.PdfFileReader(thisPath).getPage(0)
        f2=ppdf.PdfFileReader(sketchPath).getPage(0)

        f1.mergeTranslatedPage(page2=f2, tx=15, ty=245, expand=False)

        writer=ppdf.PdfFileWriter()
        writer.addPage(f1)
        with open(thisPath,'wb') as f3:
            writer.write(f3)
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()